In [14]:
!python -V

Python 3.9.20


In [15]:
import pandas as pd

In [16]:
import pickle

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [19]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment_tracking/mlruns/1', creation_time=1732838050542, experiment_id='1', last_update_time=1732838050542, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [20]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(1343254, 1340859)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [21]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [19]:
with mlflow.start_run():
   mlflow.set_tag("developer", "peter")

   mlflow.log_param("train-data", 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet')
   mlflow.log_param("valid-data",'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet')

   alpha = 0.1
   mlflow.log_param("alpha", alpha)
   lr = Lasso(alpha)
   lr.fit(X_train, y_train)

   y_pred = lr.predict(X_val)

   rmse = mean_squared_error(y_val, y_pred, squared=False)
   mlflow.log_metric("rmse", rmse)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [23]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params = params,
            dtrain=train,
            num_boost_round=50,
            evals=[(valid, "validation")],
            early_stopping_rounds=10
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)
    
    return {'loss':rmse, 'status':STATUS_OK}

In [17]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [05:00:47] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



: 

In [25]:
params = {
    'learning_rate': 0.29340071698370296,
    'max_depth': 10,
   ' min_child_weight':7.210827185243098,
    'objective':'reg:linear',
    'reg_alpha': 0.012543543806374995,
    'reg_lambda': 0.12528835322055074,
    'seed':42
}

mlflow.autolog()

with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params = params,
            dtrain=train,
            num_boost_round=50,
            evals=[(valid, "validation")],
            early_stopping_rounds=10
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)

2024/11/29 05:09:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/11/29 05:09:12 WARNING mlflow.xgboost: Failed to log dataset information to MLflow Tracking. Reason: Unable to allocate 137. GiB for an array with shape (1343254, 27306) and data type float32
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [05:09:12] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


XGBoostError: [05:09:12] /workspace/src/learner.cc:722: Invalid parameter " min_child_weight" contains whitespace.
Stack trace:
  [bt] (0) /home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x22dcbc) [0x713e8cfbecbc]
  [bt] (1) /home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x5c7316) [0x713e8d358316]
  [bt] (2) /home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x5c965a) [0x713e8d35a65a]
  [bt] (3) /home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(XGBoosterBoostedRounds+0x34) [0x713e8cec6704]
  [bt] (4) /home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/lib-dynload/../../libffi.so.8(+0xa052) [0x713f5469c052]
  [bt] (5) /home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/lib-dynload/../../libffi.so.8(+0x8925) [0x713f5469a925]
  [bt] (6) /home/codespace/anaco

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        